In [1]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(os.getenv('TF_GPU_ALLOCATOR'))

cuda_malloc_async


In [2]:
import numpy as np
import pandas as pd

airlines = pd.read_csv("/mnt/c/Users/lokes/Downloads/airlines.csv")
airlines

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [3]:
airports = pd.read_csv("/mnt/c/Users/lokes/Downloads/airports.csv")
airports

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
...,...,...,...,...,...,...,...
317,WRG,Wrangell Airport,Wrangell,AK,USA,56.48433,-132.36982
318,WYS,Westerly State Airport,West Yellowstone,MT,USA,44.68840,-111.11764
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023


In [4]:
flights = pd.read_csv("/mnt/c/Users/lokes/Downloads/flights.csv")
flights_data = flights[0:50000]
flights_data

/tmp/ipykernel_879/53272068.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  flights = pd.read_csv("/mnt/c/Users/lokes/Downloads/flights.csv")


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2015,1,4,7,AA,1524,N499AA,DFW,LAS,915,...,1010.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
49996,2015,1,4,7,AA,2316,N3FNAA,STX,MIA,915,...,1126.0,4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
49997,2015,1,4,7,AS,5,N566AS,DCA,LAX,915,...,1247.0,37.0,0,0,NaN,37.0,0.0,0.0,0.0,0.0
49998,2015,1,4,7,DL,688,N893AT,ATL,ICT,915,...,1037.0,-3.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
from pyspark import SparkContext, SparkConf
sc = SparkContext('local')
print(sc.version)

22/05/05 23:59:17 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 172.31.182.58 instead (on interface eth0)
22/05/05 23:59:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/05 23:59:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


3.2.1


In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Print my_spark
print(spark)

In [7]:
# Print the tables in the catalog
print(spark.catalog.listTables())

[]


In [8]:
# Create pd_temp
pd_temp = pd.DataFrame(np.random.random(10))

# Create spark_temp from pd_temp
spark_temp = spark.createDataFrame(pd_temp)

# Examine the tables in the catalog
print(spark.catalog.listTables())

# Add spark_temp to the catalog
spark_temp.createOrReplaceTempView('temp')

# Examine the tables in the catalog again
print(spark.catalog.listTables())

[]
[Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [9]:
# Read in the airports data
airports = spark.read.csv(['/mnt/c/Users/lokes/Downloads/airports.csv'], header = True)

# Show the data
airports.show(10)

+---------+--------------------+-------------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|         CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-------------+-----+-------+--------+----------+
|      ABE|Lehigh Valley Int...|    Allentown|   PA|    USA|40.65236| -75.44040|
|      ABI|Abilene Regional ...|      Abilene|   TX|    USA|32.41132| -99.68190|
|      ABQ|Albuquerque Inter...|  Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|     Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|       Albany|   GA|    USA|31.53552| -84.19447|
|      ACK|Nantucket Memoria...|    Nantucket|   MA|    USA|41.25305| -70.06018|
|      ACT|Waco Regional Air...|         Waco|   TX|    USA|31.61129| -97.23052|
|      ACV|      Arcata Airport|Arcata/Eureka|   CA|    USA|40.97812|-124.10862|
|      ACY|Atlantic City Int...|Atlantic City|   NJ|    USA|39.45758| -74.57717|
|      ADK|        Adak Airp

In [10]:
flight_data = spark.read.csv(['/mnt/c/Users/lokes/Downloads/flights.csv'], header = True)
flight_data = flight_data.take(100000)
#flight_data = flight_data.take(50000)

flights=spark.createDataFrame(flight_data)

# print the tables in catalog
print(spark.catalog.listTables())

# adding data into spark view for sql querying
flights.createOrReplaceTempView('flights')

# print the tables in catalog
print(spark.catalog.listTables())

22/05/05 23:59:27 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]
[Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [11]:
#flights.show()

In [12]:
# Show the data shape
print((flights.count(), len(flights.columns)))

22/05/05 23:59:40 WARN TaskSetManager: Stage 7 contains a task of very large size (13957 KiB). The maximum recommended task size is 1000 KiB.


(100000, 31)


In [13]:
# see all columns in the table 
print(flights.columns)

['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']


In [14]:
query = "SELECT AIRLINE, FLIGHT_NUMBER, TAIL_NUMBER, ORIGIN_AIRPORT, DESTINATION_AIRPORT, SCHEDULED_DEPARTURE FROM flights LIMIT 5"

flights5 = spark.sql(query)
#flights5.show()

In [15]:
query = "SELECT ORIGIN_AIRPORT, DESTINATION_AIRPORT, COUNT(*) as N FROM flights GROUP BY ORIGIN_AIRPORT, DESTINATION_AIRPORT"


flight_counts = spark.sql(query)
pd_counts = flight_counts.toPandas()

print(pd_counts.head())

22/05/05 23:59:42 WARN TaskSetManager: Stage 10 contains a task of very large size (13957 KiB). The maximum recommended task size is 1000 KiB.


  ORIGIN_AIRPORT DESTINATION_AIRPORT   N
0            BQN                 MCO  14
1            PHL                 MCO  90
2            MCI                 IAH  33
3            SPI                 ORD  19
4            SNA                 PHX  72


In [16]:
# Create the DataFrame flights
flights = spark.table("flights")

# Add duration_hrs
flights = flights.withColumn('duration_hrs', flights.AIR_TIME/60.)

# Show the head
flights.select('duration_hrs').show(10)

+------------------+
|      duration_hrs|
+------------------+
| 2.816666666666667|
| 4.383333333333334|
| 4.433333333333334|
|               4.3|
| 3.316666666666667|
| 3.433333333333333|
| 2.566666666666667|
|               3.8|
|2.8833333333333333|
|               3.1|
+------------------+
only showing top 10 rows



22/05/05 23:59:43 WARN TaskSetManager: Stage 13 contains a task of very large size (13957 KiB). The maximum recommended task size is 1000 KiB.


In [17]:
# Filter flights by passing a string
long_flights1 = flights.filter("DISTANCE > 1000")

# Filter flights by passing a column of boolean values
long_flights2 = flights.filter(flights.DISTANCE > 1000)


In [19]:
# Select the first set of columns
selected1 = flights.select('TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',)

# Select the second set of columns
temp = flights.select(flights.ORIGIN_AIRPORT, flights.DESTINATION_AIRPORT, flights.AIRLINE)

In [20]:
# Define first filter
filterA = flights.ORIGIN_AIRPORT == "SEA"

# Define second filter
filterB = flights.DESTINATION_AIRPORT == "PDX"

# Filter the data, first by filterA then by filterB
selected2 = temp.filter(filterA).filter(filterB)

In [21]:
# Define avg_speed
avg_speed = (flights.DISTANCE/(flights.AIR_TIME/60)).alias("avg_speed")

# Select the correct columns
speed1 = flights.select('TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', avg_speed)

# Create the same table using a SQL expression
speed2 = flights.selectExpr('TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', "DISTANCE/(AIR_TIME/60) as avg_speed")

In [22]:
#Cast the columns to integers
flights = flights.withColumn("MONTH", flights.MONTH.cast("integer"))
flights = flights.withColumn("DAY_OF_WEEK", flights.DAY_OF_WEEK.cast("integer"))
flights = flights.withColumn("AIR_TIME", flights.AIR_TIME.cast("integer"))
flights = flights.withColumn("DISTANCE", flights.DISTANCE.cast("double"))
flights = flights.withColumn("ARRIVAL_DELAY", flights.ARRIVAL_DELAY.cast("integer"))

In [25]:
# Find the shortest flight from PDX in terms of distance
flights.filter(flights.ORIGIN_AIRPORT == 'PDX').groupBy().min('DISTANCE')

# Find the longest flight from SEA in terms of air time
flights.filter(flights.ORIGIN_AIRPORT == 'SEA').groupBy().max('AIR_TIME')

DataFrame[max(AIR_TIME): int]

In [27]:
# Group by tailnum
by_plane = flights.groupBy("TAIL_NUMBER")

# Number of flights each plane made
by_plane.count().show(10)

# Group by origin
by_origin = flights.groupBy("ORIGIN_AIRPORT")

# Average duration of flights from PDX and SEA
by_origin.avg("AIR_TIME").show(10)

ERROR:root:KeyboardInterrupt while sending command.][Stage 16:>   (0 + 0) / 1]
Traceback (most recent call last):
  File "/home/lokesh/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/lokesh/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
import pyspark.sql.functions as F

flights = flights.withColumn("DEPARTURE_DELAY", flights.DEPARTURE_DELAY.cast("integer"))

# Group by month and dest
by_month_dest = flights.groupBy('MONTH', 'DESTINATION_AIRPORT')

# Average departure delay by month and destination
by_month_dest.avg('DEPARTURE_DELAY').show(10)

# Standard deviation of departure delay
by_month_dest.agg(F.stddev('DEPARTURE_DELAY')).show(10)

In [ ]:
print(airports.columns)

# Examine the data
print(airports.show(10))

In [ ]:
# Rename the faa column
airports = airports.withColumnRenamed("IATA_CODE", "DESTINATION_AIRPORT")

# Join the DataFrames
flights_with_airports = flights.join(airports , on = 'DESTINATION_AIRPORT', how = 'leftouter')

# Examine the new DataFrame
print(flights_with_airports.columns)
print(flights_with_airports.count())

In [ ]:
flights_with_airports.select('FLIGHT_NUMBER', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE').show(10)

In [ ]:
# Read in the airports data
airlines = spark.read.csv(['/mnt/c/Users/lokes/Downloads/airlines.csv'], header = True)

# Show the data shape
print((airlines.count(), len(airlines.columns)))

airlines.show()

# **USING MLLIB**

### **PREPROCESSING**

In [28]:
# filtering columns
model_data = flights.select('MONTH', 'DAY_OF_WEEK', 'AIRLINE', 'TAIL_NUMBER', 'DESTINATION_AIRPORT', 'AIR_TIME', 'DISTANCE', 'ARRIVAL_DELAY',)

# Remove missing values
model_data = model_data.filter("ARRIVAL_DELAY is not NULL and AIRLINE is not NULL and AIR_TIME is not NULL and TAIL_NUMBER is not NULL")

# rows left
model_data.count()

ERROR:root:KeyboardInterrupt while sending command.][Stage 16:>   (0 + 0) / 1]
Traceback (most recent call last):
  File "/home/lokesh/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/lokesh/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# Create is_late (label)
model_data = model_data.withColumn("is_late", model_data.ARRIVAL_DELAY > 0)

# cast
model_data = model_data.withColumn("is_late", model_data.is_late.cast("integer"))

# rename column
model_data = model_data.withColumnRenamed("is_late", 'label')

In [ ]:
model_data.show(15)

In [ ]:
print('Labels distrubution:')
model_data.groupBy('label').count().show()

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

# Create a StringIndexer
airline_indexer = StringIndexer(inputCol="AIRLINE", outputCol="airline_index")

# Create a OneHotEncoder
airline_encoder = OneHotEncoder(inputCol="airline_index", outputCol="airline_fact")

In [ ]:
# Create a StringIndexer
dest_indexer = StringIndexer(inputCol="DESTINATION_AIRPORT", outputCol="dest_index")

# Create a OneHotEncoder
dest_encoder = OneHotEncoder(inputCol="dest_index", outputCol="dest_fact")

In [ ]:
# Create a StringIndexer
tail_indexer = StringIndexer(inputCol="TAIL_NUMBER", outputCol="tail_index")

# Create a OneHotEncoder
tail_encoder = OneHotEncoder(inputCol="tail_index", outputCol="tail_fact")

In [ ]:
from pyspark.ml.feature import VectorAssembler

# Make a VectorAssembler of 'MONTH', 'DAY_OF_WEEK', 'AIR_TIME', 'DISTANCE', 'ARRIVAL_DELAY','AIRLINE', 'TAIL_NUMBER', 'DESTINATION_AIRPORT'
vec_assembler = VectorAssembler(inputCols=["MONTH", "DAY_OF_WEEK", "AIR_TIME", "DISTANCE", "airline_fact", "dest_fact", "tail_fact"], outputCol="features")

In [ ]:
 # Import Pipeline
from pyspark.ml import Pipeline

# Make the pipeline
flights_pipe = Pipeline(stages=[dest_indexer, dest_encoder, airline_indexer, airline_encoder, tail_indexer, tail_encoder, vec_assembler])

In [ ]:
piped_data = flights_pipe.fit(model_data).transform(model_data)

# **TRAIN - TEST SPLIT**

In [ ]:
train_data, test_data = piped_data.randomSplit([.7, .3])

In [ ]:
print('data points(rows) in train data :',  train_data.count())
print('data points(rows) in test data :',  test_data.count())

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier

import pyspark.ml.tuning as tune
from time import time
import pyspark.ml.evaluation as evals

# Create a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")

In [ ]:
start_time = time() 

# Train a LogisticRegression model
lr = LogisticRegression()

lr_model = lr.fit(train_data)

end_time = time()
elapsed_time_lr = end_time - start_time
print("Time to train LogisticRegression model: %.3f seconds" % elapsed_time_lr)

#elapsed_time_lr1 = end_time - start_time
#print("Time to train LogisticRegression model: %.3f seconds" % elapsed_time_lr1)

In [ ]:
start_time = time() 

# Train a Linear SVC model
lsvc= LinearSVC()

lsvc_model = lsvc.fit(train_data)

end_time = time()
elapsed_time_lsvc = end_time - start_time
print("Time to train LinearSVC model: %.3f seconds" % elapsed_time_lsvc)

#elapsed_time_lsvc1 = end_time - start_time
#print("Time to train LinearSVC model: %.3f seconds" % elapsed_time_lsvc1)

In [ ]:
start_time = time() 

# Train a DecisionTree model
dt = DecisionTreeClassifier()

model_dt = dt.fit(train_data)

end_time = time()
elapsed_time_dt = end_time - start_time
print("Time to train DecisionTree model: %.3f seconds" % elapsed_time_dt)

#elapsed_time_dt1 = end_time - start_time
#print("Time to train DecisionTree model: %.3f seconds" % elapsed_time_dt1)

In [ ]:
start_time = time() 

# Train a RandomForest model
rf = RandomForestClassifier() 

model_rf = rf.fit(train_data)

end_time = time()
elapsed_time_rf = end_time - start_time
print("Time to train RandomForest model: %.3f seconds" % elapsed_time_rf)

#elapsed_time_rf1 = end_time - start_time
#print("Time to train RandomForest model: %.3f seconds" % elapsed_time_rf1)

# **LOGISTIC REGRESSION**

In [ ]:
# Import LogisticRegression
from pyspark.ml.classification import LogisticRegression

# Create a LogisticRegression Estimator
lr = LogisticRegression()

In [ ]:
# Import the evaluation submodule
import pyspark.ml.evaluation as evals

# Create a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")

In [ ]:
# Import the tuning submodule
import pyspark.ml.tuning as tune

# Create the parameter grid
grid1 = tune.ParamGridBuilder()

# Add the hyperparameter
grid1 = grid1.addGrid(lr.regParam, np.arange(0, .1, .01))
grid1 = grid1.addGrid(lr.elasticNetParam, [0, 1])

# Build the grid
grid1 = grid1.build()

In [ ]:
# Create the CrossValidator
cv = tune.CrossValidator(estimator=lr,
               estimatorParamMaps=grid1,
               evaluator=evaluator)

In [ ]:
# Call lr.fit()
best_lr = lr.fit(train_data)

# Print best_lr
print(best_lr)

In [ ]:
# Use the model to predict the test set
test_results = best_lr.transform(test_data)

# Evaluate the predictions
print(evaluator.evaluate(test_results))

# **LINEAR SVC**

In [ ]:
from pyspark.ml.classification import LinearSVC

lsvc= LinearSVC()

In [ ]:
# Import the evaluation submodule
import pyspark.ml.evaluation as evals

# Create a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")

In [ ]:
# Import the tuning submodule
import pyspark.ml.tuning as tune

# Create the parameter grid
grid2 = tune.ParamGridBuilder()

# Add the hyperparameter
grid2 = grid2.addGrid(lsvc.regParam, np.arange(0, .1, .01))


# Build the grid
grid2 = grid2.build()

In [ ]:
# Create the CrossValidator
cv = tune.CrossValidator(estimator=lsvc,
               estimatorParamMaps=grid2,
               evaluator=evaluator)

In [ ]:
# Call lsvc.fit()
best_lsvc = lsvc.fit(train_data)

# Print best_lr
print(best_lsvc)

In [ ]:
# Use the model to predict the test set
test_results = best_lsvc.transform(test_data)

# Evaluate the predictions
print(evaluator.evaluate(test_results))

# **DECISION TREE**

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier()


# Import the evaluation submodule
import pyspark.ml.evaluation as evals

# Create a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")

# Import the tuning submodule
import pyspark.ml.tuning as tune

# Create the parameter grid
grid3 = tune.ParamGridBuilder()

# Build the grid
grid3 = grid3.build()

# Create the CrossValidator
cv = tune.CrossValidator(estimator=dt,
               estimatorParamMaps=grid3,
               evaluator=evaluator)


# Call dt.fit()
best_dt = dt.fit(train_data)

# Print best_dt
print(best_dt)


# Use the model to predict the test set
test_results = best_dt.transform(test_data)

# Evaluate the predictions
print(evaluator.evaluate(test_results))

# **RANDOM FOREST CLASSIFIER**

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
import pyspark.ml.tuning as tune
from time import time
import pyspark.ml.evaluation as evals


# Create a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")


rf = RandomForestClassifier()


# Create the parameter grid
grid4 = tune.ParamGridBuilder()

# Build the grid
grid4 = grid4.build()

# Create the CrossValidator
cv = tune.CrossValidator(estimator=rf,
               estimatorParamMaps=grid4,
               evaluator=evaluator)


# Call rf.fit()
start_time = time()

best_rf = rf.fit(train_data)

end_time = time()
elapsed_time = end_time - start_time
print("Time to train model: %.3f seconds" % elapsed_time)

# Print best_rf
print(best_rf)



In [ ]:
# Use the model to predict the test set
s = time()
test_results = best_rf.transform(test_data)
e= time()
elapsed_time = e-s
print("Time to test model: %.3f seconds" % elapsed_time)

# Evaluate the predictions
print(evaluator.evaluate(test_results))

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from time import time

start_time = time() 

# Train a RandomForest model
rf = RandomForestClassifier(labelCol="label", 
     featuresCol="features", 
     numTrees=500,
     maxDepth=3, 
     seed = 1,
     featureSubsetStrategy="sqrt",
     impurity='gini') 

model = rf.fit(train_data)

end_time = time()
elapsed_time = end_time - start_time
print("Time to train model: %.3f seconds" % elapsed_time)

In [ ]:
# Create the parameter grid
grid4 = tune.ParamGridBuilder()

# Build the grid
grid4 = grid4.build()

# Create the CrossValidator
cv = tune.CrossValidator(estimator=rf,
               estimatorParamMaps=grid4,
               evaluator=evaluator)


# Call rf.fit()
best_rf = rf.fit(train_data)

# Print best_rf
print(best_rf)

# Use the model to predict the test set
test_results = best_rf.transform(test_data)

# Evaluate the predictions
print(evaluator.evaluate(test_results))